In [ ]:
reload("Utils")

In [1]:
using ProgressMeter
using SwiftObjectStores
using CorpusLoaders
using WordNet
using AdaGram
using AdaGramCompat
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using JLD

using SenseAlignment

importfrom(CorpusLoaders, :sensekey)
importfrom(WordNet, :sensekey)

In [3]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [4]:
#am = AdaGramCompat.AdaGramModel(load_model("models/adagram/v1_d100.adagram_model")...)
#am = AdaGramModel(load_model("models/adagram/more_senses.adagram_model")...)
#am = open(deserialize,"models/adagram/more_senses.adagram_model.jsz", "r");
am = load("models/adagram/more_senses.adagram_model.jld", "am");


In [ ]:
s_am = get_jld(SwiftService(), "sensemodels", "adagram/semhuff_more_senses.adagram_model.jld", "am");

In [276]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld");
ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld", "ee")

#ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

In [5]:
challenges = load_challenges_semeval2007t7("data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml");
solutions = load_solutions_semeval2007t7("data/corpora/wsd/semeval2007_t7/key/dataset21.test.key");
semcor = index_semcor(lazyload_semcor("data/corpora/semcor2.1/brown1/tagfiles/"));

In [6]:
function window(context, index::Int, window_size::Int=10)
    window_lower_bound = max(index - window_size÷2, 1)
    window_upper_bound = min(index + window_size÷2, length(context))
    view(context, [window_lower_bound:index-1 ; index+1:window_upper_bound])
end

function window(tagged_sense::TaggedSentence, index::Int, window_size::Int=10)
    context = lowercase.(strip_tags(tagged_sense))
    window(context, index, window_size)
end
   
function window(context, word::AbstractString, window_size::Int=10)
    context = lowercase.(context)
    occurances = find(context.==word)
    index =  length(occurances) > 0 ? occurances[ceil(Int, end/2)] : 0
    window(context, index, window_size)

end


window((drop(semcor,400) |> first |> last |> first)...)

window((synsets(db, db["fireplace", 'n']) |> first).gloss |> split, "fireplace")

10-element SubArray{String,1,Array{String,1},Tuple{Array{Int64,1}},false}:
 "fire"  
 "can"   
 "be"    
 "built;"
 "\"the" 
 "was"   
 "so"    
 "large" 
 "you"   
 "could" 

In [22]:

"Collect up the usages from a indexed tagged source"
function get_usages(usage_index::SemcorIndex, key)
    if haskey(usage_index, key)
        [window(lowercase.(strip_tags(sent)), index) for (sent, index) in usage_index[key]]
    else
        Vector{String}[]
    end
end

function get_usages(synset::Synset, lemma_word::AbstractString)
    gloss::Vector{SubString{String}} = lowercase.(punctuation_space_tokenize(synset.gloss))
    [window(gloss, lemma_word)]
end

function get_all_usages(wn::DB, semcor::SemcorIndex, lemma_word, pos)   
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    
    Dict{Synset,AbstractVector{AbstractVector}}((synset => get_usages(synset, lemma_word)
        #[get_usages(semcor, sensekey(db, synset, lemma)); get_usages(synset, lemma_word)]
                    for synset in target_synsets))  
    
end

get_all_usages (generic function with 1 method)

In [23]:
all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])

function _lexically_informed_embeddings(wn::DB,ee, word,lemma_word, pos)
    indexed_corpus = semcor #TODO: Pass this as a parameter, not as a gloel
    target_synset_examples = get_all_usages(wn, indexed_corpus, lemma_word, pos)
    target_synsets = collect(keys(target_synset_examples))
    lem = db[lemma_word, pos]
        
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,(synset, examples)) in enumerate(target_synset_examples)
        context::Vector{SubString{String}} = vcat(examples...)
        embeddings[ii] =synthesize_embedding(ee,context, word, lemma_word)
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sensecount(db, ss, lem ) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lem
end

lexically_informed_embeddings(wn::DB, ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
        
_li_embeddings = Dict{Tuple{DB, Any, String, String, Char}, Tuple{Vector{Vector{Float32}}, Vector{Synset}, Lemma}}()
function lexically_informed_embeddings(wn::DB, ee, word,lemma_word, pos)
    get!(_li_embeddings, (wn, ee, word, lemma_word, pos)) do
        _lexically_informed_embeddings(wn, ee, word, lemma_word, pos)
    end
end

lexically_informed_embeddings (generic function with 2 methods)

In [89]:
function supervised_wsd(challenge, ee, wn::DB)
    try
        embeddings,target_synsets,lem = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        
        priors = [float(sensecount(db, ss, lem)) for ss in target_synsets] #TODO use thing
        priors .+= 100
        #priors .+= sqrt(sum(priors))/length(priors)
        priors ./= sum(priors)
        probs_of_sense = general_wsd(ee, challenge.context, embeddings, priors)
               
     
        sense_index= indmax(probs_of_sense)        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        @show ex
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB)
    try
        lem = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lem)
        
        sense_freqs =  Float32[sensecount(db, ss, lem) for ss in target_synsets]
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sensekey(wn, synset,  lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 1 method)

In [90]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    @showprogress for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    @show notattempted
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB)
    method = challenge -> supervised_wsd(challenge, ee, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB)
    method = challenge -> most_frequent_sense(challenge, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [95]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db)

Progress:  19%|████████                                 |  ETA: 0:00:08ex = KeyError(" SubString{String}[\"noncompetitively\"], nor SubString{String}[\"noncompetitively\"] have embeddings")
Progress:  23%|██████████                               |  ETA: 0:00:08ex = KeyError(" SubString{String}[\"semiliterate\"], nor SubString{String}[\"semiliterate\"] have embeddings")
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
notattempted = 2
overall: 		(0.7953242170269078,0.7946231820185103,0.794973544973545)


In [370]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db)

ex = KeyError(" SubString{String}[\"aftereffects\"], nor SubString{String}[\"aftereffect\"] have embeddings")
Progress:   4%|██                                       |  ETA: 0:00:02ex = KeyError(" SubString{String}[\"predispose\"], nor SubString{String}[\"predispose\"] have embeddings")
Progress:  12%|█████                                    |  ETA: 0:00:04ex = KeyError(" SubString{String}[\"insinuating\"], nor SubString{String}[\"insinuate\"] have embeddings")
Progress:  14%|██████                                   |  ETA: 0:00:04ex = KeyError(" SubString{String}[\"high-minded\"], nor SubString{String}[\"high-minded\"] have embeddings")
ex = KeyError(" SubString{String}[\"scammers\"], nor SubString{String}[\"scammer\"] have embeddings")
Progress:  19%|████████                                 |  ETA: 0:00:03ex = KeyError(" SubString{String}[\"noncompetitively\"], nor SubString{String}[\"noncompetitively\"] have embeddings")
Progress:  23%|█████████                                |  ETA

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, s_am, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), s_am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), s_am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), s_am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), s_am, db)

# "models/adagram/more_senses.adagram_model.jld
### with Adagram Disambig weighting (prior used)
### with SemCor data 
overall: 		(0.655408489274304,0.6328779197884531,0.6439461883408072)
- nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7161410018552876,0.6967509025270758,0.706312900274474)
- vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5106382978723404,0.4873096446700508,0.4987012987012987)
- aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6619718309859155,0.649171270718232,0.6555090655509065)
- rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7268041237113402,0.6778846153846154,0.7014925373134329)

### with Adagram Disambig weighting (prior used)
### with just glosses 

overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
- nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
- vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
- aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
- rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)



# "semhuff_more_senses.adagram_model.jld

### with Adagram Disambig weighting (prior used)
### with just glosses 
 - overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
 - nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
 - vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
 - aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
 - rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)
 
 ### with Adagram Disambig weighting (prior used)
 ### With semcore
 - overall: 		(0.6134185303514377,0.5923314235345968,0.6026905829596412)
 - nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),s_am,db) = (0.6252319109461967,0.6083032490974729,0.6166514181152789)
 - vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),s_am,db) = (0.5638297872340425,0.5380710659898477,0.5506493506493505)
 - aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),s_am,db) = (0.6169014084507042,0.6049723756906077,0.610878661087866)
 - rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),s_am,db) = (0.6855670103092784,0.6394230769230769,0.6616915422885572)
 
 
 # Most Frequent Sense
 
  -  overall: 		(0.7783164389598942,0.7783164389598942,0.7783164389598942)
  - only nouns  : 		(0.7653429602888087,0.7653429602888087,0.7653429602888087)
  - only verbs  : 		(0.7529610829103215,0.7529610829103215,0.7529610829103215)
  - only adjecti: 		(0.8038674033149171,0.8038674033149171,0.8038674033149171)
  - only adverbs: 		(0.875,0.875,0.875)


In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

#println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db))
#println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db))
#println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db))
#println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db))

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db))

#println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db))
#println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db))
#println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db))
#println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db))


In [ ]:
#Zero shot WSI for 1 shot WSD
using Training

In [ ]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=50)
rr.corpus_size = ee.corpus_size
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr);

In [ ]:
rr.embedding["us"] |> length

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db))

In [ ]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db))

In [119]:
mapping_corpus = load_semcor("data/corpora/semcor2.1/brown1/tagfiles/");

In [400]:
function identify(taggedword::TaggedWord, wn::DB,ee)
    pos = pennPOStoWordNetPOS(taggedword.pos)
    wn_sensekeys = sensekeys(wn, wn[pos, taggedword.lemma])
    nsenses = length(wn_sensekeys)
    
    target_wnsn = findfirst(wn_sensekeys .== sensekey(taggedword))
    target_wnsn==0 && throw(KeyError("$(sensekey(taggedword)) not in  $(wn_sensekeys).\n\nIssue is with $taggedword"))

    wvs = all_word_sense_vectors(ee,taggedword.word)
    length(wvs) == 0 && throw(KeyError("No embedding for $(taggedword.word); skipping"))
    
    target_wnsn, nsenses, pos, wvs
end

identify (generic function with 1 method)

In [409]:
function agirresAlignment(ee, wn::DB, mapping_corpus, hard=true)
    maps = Dict{Tuple{String, String, Char}, Matrix{Float32}}()
    freqs = Dict{Tuple{String, String, Char}, Vector{Int}}()
    
    function proc_word(word::TaggedWord, sentence::TaggedSentence)
    end
    
    function proc_word(taggedword::SenseAnnotatedWord, sentence::TaggedSentence)
        local target_wnsn, nsenses, pos, wvs
        try
            target_wnsn, nsenses, pos, wvs = identify(taggedword, wn::DB,ee)
        catch err
            typeof(err)<:KeyError ||rethrow(ee)
            return
        end
        ########
        
        map = get!(maps, (taggedword.word, taggedword.lemma, pos)) do
            zeros(length(wvs), nsenses)
        end
        
        freq = get!(freqs, (taggedword.word, taggedword.lemma, pos)) do 
            zeros(Int, nsenses)
        end
        
        
        context = lowercase.(strip_tags(sentence))
        wv_probs = general_wsd(ee,context, wvs)
        @assert(length(wv_probs) == length(wvs))
        @assert sum(wv_probs) ≈ 1f0
        @assert !any(isnan.(wv_probs))
        freq[target_wnsn] += 1
        if hard
            map[indmax(wv_probs),target_wnsn] += 1
        else
            @assert(length(map[:,target_wnsn]) == length(wv_probs),
            "$(length(map[:,target_wnsn])) != $(length(wv_probs)) for \"$(taggedword.lemma)\""           
            )
            map[:,target_wnsn] += wv_probs
        end
        
    end
    
    @showprogress for sentence in mapping_corpus
        for word in sentence
            proc_word(word, sentence)
        end
    end
 
    
    @showprogress for ((word, lem, pos), freq) in freqs
        mm = maps[(word, lem, pos)]
        mm ./= freq'
        mm[isnan.(mm)] = 0f0 #NaNs are just frequency 0 items
        @assert(all(isapprox.(sum(mm, 1), 1f0; atol=1f-5) 
                    | isapprox.(sum(mm, 1), 0f0; atol=1f-5)), 
                    "($word , $pos) not sum to one, $(sum(mm,1))")
    end

    maps, freqs
end

agirresAlignment (generic function with 2 methods)

In [466]:
function mapped_wsd(challenge, ee, wn::DB, maps::Associative{Tuple{String, String, Char}, Matrix{Float32}})
    try
        lem = wn[challenge.lemma, challenge.pos]
        target_synsets = synsets(wn, lem)
        
        wvs = all_word_sense_vectors(ee, challenge.word)
        length(wvs) > 0 || throw(KeyError("No embeddings for "*challenge.word))
        
        wv_probs = general_wsd(ee,challenge.context, wvs)

        
        mm = copy(maps[challenge.word, challenge.lemma, challenge.pos])
        mm.+=1.0
        probs_of_sense = mm'*wv_probs
        probs_of_sense./=sum(probs_of_sense)

        
        sense_index= indmax(probs_of_sense)        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        most_frequent_sense(challenge, wn) #Fallback on MFS
    end
end

function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB, maps)
    method = challenge -> mapped_wsd(challenge, ee, wn, maps)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge (generic function with 3 methods)

In [411]:
function AdaGramCompat.all_word_sense_vectors(ee::WordSenseEmbedding, word)
    get(ee.embedding, word, Vector{Float32}[])
end

In [467]:
maps_hard, freqs  = agirresAlignment(am, db, mapping_corpus, true);

Progress: 100%|█████████████████████████████████████████| Time: 0:31:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


In [468]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db, maps_soft))

#maps_soft(am) overall: 		(0.6897311591009255,0.6897311591009255,0.6897311591009255

Progress: 100%|█████████████████████████████████████████| Time: 0:00:49
notattempted = 0
overall: 		(0.6897311591009255,0.6897311591009255,0.6897311591009255)


In [469]:
function matrix_from_rows{T}(Xs::Vector{Vector{T}})
    ret = Matrix{T}(length(Xs), length(first(Xs)))
    for (ii,row) in enumerate(Xs)
        @inbounds ret[ii,:] = row
    end
    ret
end

function bestfact!(x)
    if size(x,1)<size(x,2)
        svdfact!(x)
    elseif size(x,1)>=size(x,2)
        qrfact!(x, Val{true})
    #else
    #    @assert(size(x,1)==size(x,2))
    #    lufact!(x)
    end
end

function leastSquaresAlignment(ee, wn::DB, mapping_corpus, hard=true)
    vars = Dict{Tuple{String, String, Char}, Tuple{Vector{Vector{Float32}}, Vector{Vector{Float32}}}}()
    
    function proc_word(word::TaggedWord, sentence::TaggedSentence)
    end
    
    function proc_word(taggedword::SenseAnnotatedWord, sentence::TaggedSentence)
        local target_wnsn, nsenses, pos, wvs
        try
            target_wnsn, nsenses, pos, wvs = identify(taggedword, wn::DB,ee)
        catch err
            typeof(err)<:KeyError ||rethrow(ee)
            return
        end
        ########
        
        Xs, Ys = get!(vars, (taggedword.word, taggedword.lemma, pos)) do
            Vector{Vector{Float32}}(), Vector{Vector{Float32}}()
        end    
        
        context = lowercase.(strip_tags(sentence))
        wv_probs = general_wsd(ee,context, wvs)
        @assert(length(wv_probs) == length(wvs))
        @assert sum(wv_probs) ≈ 1f0
        @assert !any(isnan.(wv_probs))
        push!(Xs, wv_probs)
        y = zeros(nsenses)
        y[target_wnsn] = 1f0
        push!(Ys, y)
    end
    
    @showprogress for sentence in mapping_corpus
        for word in sentence
            proc_word(word, sentence)
        end
    end
 
    maps = Dict{Tuple{String, String, Char}, Matrix{Float32}}()
    
    @showprogress for (key, (Xs,Ys)) in vars
        X = matrix_from_rows(Xs)
        Y = matrix_from_rows(Ys)
        try 
            maps[key] = bestfact!(X)\Y
        catch err
            @show X
            println("-"^30)
            @show Y
            rethrow(err)
        end    
    end

    maps
end

leastSquaresAlignment (generic function with 2 methods)

In [ ]:
maps_ls_am = leastSquaresAlignment(am, db, mapping_corpus);

Progress:   9%|████                                     |  ETA: 0:23:18

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db, maps_ls_am))
#am, db, maps_ls_am overall: 		(0.753195240193918,0.753195240193918,0.7531952401939179)

In [462]:
mm=drop(maps_ls_am,5) |> first |> last
mm+=1
mm./=sum(mm,1)

30×3 Array{Float32,2}:
 0.0333333  0.0249207  0.0333333
 0.0333333  0.0316945  0.0333333
 0.0333333  0.0217652  0.0333333
 0.0333333  0.0187715  0.0333333
 0.0333333  0.0274698  0.0333333
 0.0333333  0.0310764  0.0333333
 0.0333333  0.0208254  0.0333333
 0.0333333  0.0256236  0.0333333
 0.0333333  0.0362388  0.0333333
 0.0333333  0.0363548  0.0333333
 0.0333333  0.0362836  0.0333333
 0.0333333  0.036307   0.0333333
 0.0333333  0.0362278  0.0333333
 0.0333333  0.0360908  0.0333333
 0.0333333  0.0361495  0.0333333
 0.0333333  0.0360063  0.0333333
 0.0333333  0.0361296  0.0333333
 0.0333333  0.0361088  0.0333333
 0.0333333  0.0364005  0.0333333
 0.0333333  0.0364899  0.0333333
 0.0333333  0.0363374  0.0333333
 0.0333333  0.0365099  0.0333333
 0.0333333  0.0364396  0.0333333
 0.0333333  0.0362374  0.0333333
 0.0333333  0.0364378  0.0333333
 0.0333333  0.0362699  0.0333333
 0.0333333  0.0362849  0.0333333
 0.0333333  0.0362905  0.0333333
 0.0333333  0.0360832  0.0333333
 0.0333333  0.036174

In [ ]:
function hybrid_wsd(challenge, ee, wn::DB, maps::Associative{Tuple{String, String, Char}, Matrix{Float32}})
    try
        lem = wn[challenge.lemma, challenge.pos]
        target_synsets = synsets(wn, lem)
        
        wvs = all_word_sense_vectors(ee, challenge.word)
        length(wvs) > 0 || throw(KeyError("No embeddings for "*challenge.word))
        
        wv_probs = general_wsd(ee,challenge.context, wvs)

        
        mm = copy(maps[challenge.word, challenge.lemma, challenge.pos])
        mm.+=1.0
        mm/=sum(mm,1)
        probs_of_sense = mm'*wv_probs
        probs_of_sense./=sum(probs_of_sense)

        
        sense_index= indmax(probs_of_sense)        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        most_frequent_sense(challenge, wn) #Fallback on MFS
    end
end

function evalute_on_wsd_challenge_hybrid(challenges, solutions, ee, wn::DB, maps)
    method = challenge -> hybrid_wsd(challenge, ee, wn, maps)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge_hybrid(challenges, solutions, am, db, maps_ls_am))
#am, db, maps_ls_am overall: 		(0.753195240193918,0.753195240193918,0.7531952401939179)